In [ ]:
import pandas as pd
egd10 = pd.read_excel("data/EGD10.xlsx")

In [ ]:
targets = list(map(lambda x: x.upper(), [
    "PV1ling",
    "PV2ling",
    "PV3ling",
    "PV4ling",
    "PV5ling",
    "PV1mat",
    "PV2mat",
    "PV3mat",
    "PV4mat",
    "PV5mat",
    "PV1mfis",
    "PV2mfis",
    "PV3mfis",
    "PV4mfis",
    "PV5mfis",
    "PV1syc",
    "PV2syc",
    "PV3syc",
    "PV4syc",
    "PV5syc",
]))

new_targets = [
    "S15A", "S15B", "S15C", "S15D", "S15E", "S15F", "S15G",
    "S20A", "S20B", "S20C", "S20D", "S20E", "S20F", "S20G", "S20H", "S20I", "S20J",
    "S21A", "S21B", "S21C",
    "S22A", "S22B", "S22C", "S22D", "S22E", "S22F"
]

In [ ]:
egd10.loc[:, "Nota_Media"] = egd10[targets].mean(axis=1)
egd10.drop(targets, axis=1, inplace=True)
egd10.loc[:, "Nota_Media"] = egd10["Nota_Media"].apply(lambda x: x / 100)

In [ ]:
egd10.dropna(inplace=True)
# Cambiar los valores 5 por 1 en las columnas que comienzan con "S15"
for col in new_targets:
    if col.startswith("S15"):
        egd10[col] = egd10[col].replace(5, 1)
# Eliminar valores nulos
egd10 = egd10[(egd10[new_targets] != 9).all(axis=1)]

In [ ]:
# Ver distribución de valores para las variables new_targets
for col in new_targets:
    if col in egd10.columns:
        print(f"Distribución de valores para {col}:")
        print(egd10[col].value_counts(dropna=False))
        print("\n")
    else:
        print(f"La columna {col} no se encuentra en el DataFrame.\n")

In [ ]:
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

# Seleccionar las características para calcular la matriz de correlación
df_feat = egd10[["Nota_Media"] + new_targets]

# Calcular la matriz de correlación
corr = df_feat.corr()

# Generar una máscara para el triángulo superior
mask = np.triu(np.ones_like(corr, dtype=bool))

# Configurar la figura de matplotlib
plt.figure(figsize=(11, 9))

# Generar un colormap divergente personalizado
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Dibujar el heatmap con la máscara y la relación de aspecto correcta
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

# Calcular la matriz de distancias
dist_matrix = pdist(corr)

# Realizar el enlace jerárquico
Z = linkage(dist_matrix, method='ward')

# Dibujar el dendrograma
plt.figure(figsize=(15, 10))
dendrogram(Z, labels=corr.columns, leaf_rotation=90, leaf_font_size=10)
plt.title('Dendrograma de las variables')
plt.xlabel('Variables')
plt.ylabel('Distancia')
plt.show()

In [ ]:
# Mapeos para las variables de tiempo y frecuencia
# Los valores representan horas estimadas para cada categoría
map_tiempo = {1: 0, 2: 0.25, 3: 0.75, 4: 1.5, 5: 2.5}
map_frecuencia = {1: 0, 2: 0.2, 3: 0.65, 4: 1.25}



# Listas de características según tipo de actividad
# Variables relacionadas con actividades digitales
tiempo_features_digital = ["S15D", "S15E", "S20I", "S20J"]
frecuencia_features_digital = ["S21A","S22D", "S22F", "S22E"]

# Variables relacionadas con actividades culturales
tiempo_features_cultural = ["S15C", "S15A", "S20D"]

# Conversión de variables digitales
# Mapear las categorías numéricas a horas estimadas para cada variable de tiempo y frecuencia digital
for feature in tiempo_features_digital:
    egd10[feature + "_horas_digital"] = egd10[feature].map(map_tiempo)

for feature in frecuencia_features_digital:
    egd10[feature + "_horas_digital"] = egd10[feature].map(map_frecuencia)

# Sumar todas las horas digitales en una nueva columna unificada
# Combina tiempo y frecuencia para calcular un total de horas invertidas en actividades digitales
egd10["Horas_Digitales"] = egd10[[feature + "_horas_digital" for feature in tiempo_features_digital + frecuencia_features_digital]].sum(axis=1)

# Conversión de variables culturales
# Mapear las categorías numéricas a horas estimadas para cada variable de tiempo y frecuencia cultural
# Sumar todas las horas culturales en una nueva columna unificada
# Combina tiempo y frecuencia para calcular un total de horas invertidas en actividades culturales
egd10['Total_Horas_Cultural'] = egd10[["S15C", "S15A", "S20D"]].applymap(map_tiempo.get).sum(axis=1) + egd10[["S21A", "S22D", "S22F", "S22E"]].applymap(map_frecuencia.get).sum(axis=1)



In [ ]:
# Definir el umbral de diferencia
umbral = 3.0  # Ajusta este valor según la sensibilidad deseada

# Clasificar en tres categorías
egd10['Mayor_Horas'] = egd10.apply(
    lambda row: (
        'Cultural' if row['Total_Horas_Cultural'] > row['Total_Horas_Digital'] + umbral else
        'Digital' if row['Total_Horas_Digital'] > row['Total_Horas_Cultural'] + umbral else
        'Igualado'
    ),
    axis=1
)
# Mapear las categorías de 'Mayor_Horas' a valores numéricos
egd10['Mayor_Horas_Bin'] = egd10['Mayor_Horas'].apply(
    lambda x: 1 if x == 'Cultural' else -1 if x == 'Digital' else 0
)

# Verificar las frecuencias de los valores binarios
print(egd10['Mayor_Horas_Bin'].value_counts())


# Verificar las frecuencias de cada categoría
print(egd10['Mayor_Horas'].value_counts())
